<a href="https://colab.research.google.com/github/robertobiero/selfAttentionModule/blob/main/SelfAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch





inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2


class selfAttention(torch.nn.Module):
  def __init__(self, d_in, d_out):
    super().__init__(). # run init method from parent class torch.nn.module
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

  def forward(self, x):
    Q = self.W_query(x). # linear projection of embedding x to query space
    K = self.W_key(x)
    V = self.W_value(x)

    attn_scores = Q @ K.T # compute attention scores

    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

    context_vec = attn_weights @ values
    return context_vec


torch.manual_seed(789).  # randomly initialise W-q, W_k, W_v
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))






Q_journey tensor([0.4306, 1.4551])
K_journey tensor([0.4433, 1.1419])
V_journey tensor([0.3951, 1.0037])
tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])
attention weights tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
Value vector tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])
tensor([0.3061, 0.8210])
